In [6]:
# Importing Libraries
import numpy as np
import pandas as pd

In [7]:
from helpers import BotScrap

## Here I will trying to filter data from FBREF

In [8]:
data = BotScrap()
match = data.get_match('https://fbref.com/en/matches/2e4383ca/Arsenal-Leeds-United-April-1-2023-Premier-League')

In [30]:
home = match['home'][2]
home.rename(columns={'': 'Player ID'})

,Player,#,Nation,Pos,Age,Min,Cmp,Att,Cmp%,TotDist,...,Cmp%,Ast,xAG,xA,KP,1/3,PPA,CrsPA,PrgP,Player ID
0,Gabriel Jesus,9.0,br BRA,FW,25-363,59,27,31,87.1,340,...,0.0,0,0.1,0.0,1,0,0,0,0,b66315ae
1,Bukayo Saka,7.0,eng ENG,RW,21-208,31,9,14,64.3,131,...,50.0,0,0.0,0.0,0,1,0,0,0,bc7dc64d
2,Martinelli,11.0,br BRA,LW,21-287,90,34,43,79.1,552,...,100.0,1,0.9,0.4,2,2,2,2,3,48a5a5d6
3,Leandro Trossard,19.0,be BEL,"RW,FW",28-118,90,35,43,81.4,648,...,85.7,1,1.3,0.7,3,2,0,0,2,38ceb24a
4,Granit Xhaka,34.0,ch SUI,LM,30-186,85,49,53,92.5,798,...,100.0,0,0.3,0.3,1,3,3,1,7,e61b8aee
5,Thomas Partey,5.0,gh GHA,CM,29-292,59,44,46,95.7,902,...,100.0,0,0.0,0.0,0,7,0,0,3,529f49ab
6,Jorginho,20.0,it ITA,CM,31-102,31,27,31,87.1,428,...,100.0,0,0.0,0.0,0,0,1,0,3,45db685d
7,Martin Ødegaard,8.0,no NOR,RM,24-105,84,45,54,83.3,774,...,50.0,1,0.2,0.3,2,5,1,0,4,79300479
8,Oleksandr Zinchenko,35.0,ua UKR,LB,26-107,84,88,109,80.7,1413,...,77.8,0,0.0,0.0,0,5,0,0,6,51cf8561
9,Gabriel Dos Santos,6.0,br BRA,CB,25-103,90,77,84,91.7,1277,...,80.0,0,0.0,0.0,0,8,0,0,9,67ac5bb8
